# Webcam Image

The following cells demonstrat how to access your webcam with python.

## Accessing the webcam with `imageio`

To access the webcam with `imageio`, the module `imageio-ffmpeg` has to be installed. If you used our environment (`cv.yml`), this should already be the case. You can check this by executing the following cell:

In [ ]:
import importlib
assert importlib.util.find_spec("imageio_ffmpeg"), "imageio_ffmpeg is not installed!"

If `imageio-ffmpeg` is not installed, you may need to install by typing `conda install -c conda-forge imageio-ffmpeg`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import imageio.v2 as imageio

try:
    webcam = imageio.get_reader('<video0>')
    frame = webcam.get_next_data()
    webcam.close()

    if frame is not None:
        plt.figure()
        plt.title(f"Webcam image: {frame.shape}")
        plt.imshow(frame)
        plt.show()
    else:
        print("No image.")
except Exception as ex:
    print("Error:", ex)

* a webcam can be accessed as an imageio `Reader`
* such a reader allows to read multiple images
* it is essential to close a `Reader` after using it
  - otherwise it will be blocked, prohibiting access by other programs
  - if you lost the handle (`webcam`) you cannot close it anymore
  - you may always restart the kernel to close the webcam
* you may use Python context manage (`with` block) to automatically close the `Reader`

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import imageio

try:
    with imageio.get_reader('<video0>') as webcam:
        frame = webcam.get_next_data()
    if frame is not None:
        plt.figure()
        plt.title(f"Webcam image: {frame.shape}")
        plt.imshow(frame)
        plt.show()
    else:
        print("No image.")
except Exception as ex:
    print("Error:", ex)

You can also inspect some metadata of the camera:

In [ ]:
import imageio

try:
    with imageio.get_reader('<video0>') as webcam:
        print(webcam.get_meta_data())
except Exception as ex:
    print("Error:", ex)

## Alternative: OpenCV (module `cv2`)

OpenCV provides an alternative method to access the webcam:
* `VideoCapture` object (similar to `imageio` `Reader`)
* OpenCV has to be compiled with ffmpeg support to provide webcam access
* OpenCV uses BGR (not RGB) images
  - you need to switch the color channels for displaying the image

In [ ]:
import matplotlib.pyplot as plt
import cv2
import logging

try:
    webcam = cv2.VideoCapture(0)
    ret, frame_bgr = webcam.read()
    webcam.release()

    if ret:
        frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        plt.figure()
        plt.title(f"Webcam image: {frame.shape}")
        plt.imshow(frame)
        plt.show()
    else:
        logging.error("Reading image failed!")
except Exception as ex:
    print("Error:", ex)

With OpenCV you can also inspect (and set) some camera properties:

In [ ]:
import cv2

webcam= None
try:
    webcam = cv2.VideoCapture(0)
    if not webcam.isOpened():
        raise RuntimeError(f"Error opening OpenCV VideopCapture ({index})")

    # Output camera information
    print("Frame width:", webcam.get(cv2.CAP_PROP_FRAME_WIDTH))
    print("Frame height:", webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print("Auto exposure:", webcam.get(cv2.CAP_PROP_AUTO_EXPOSURE))
    print("Exposure:", webcam.get(cv2.CAP_PROP_EXPOSURE))
    
finally:
    if webcam is not None:
        webcam.release()
        webcam = None

## Webcam Stream

A stream is essentially a sequence of images (frames).
* we can show a camera stream by repeatedly updating the display:
  - read new image from camera
  - display the image
* in `notebook` mode, a matplotlib figure can be updated and redrawn
* to stop the loop, press <kbd>I</kbd>, <kbd>I</kbd> (Kernel Interrupt)

In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import imageio.v2 as imageio

try:
    display(HTML("press <kbd>I</kbd>, <kbd>I</kbd> (Kernel Interrupt) to stop the demo!"))
    with imageio.get_reader('<video0>') as webcam:
        fig = plt.figure(figsize=(8,6))
        mpl_image = plt.imshow(webcam.get_next_data())

        while True:
            img = webcam.get_next_data()
            mpl_image.set_data(img)
            fig.canvas.draw()
except KeyboardInterrupt:
    print("Interrupted")
finally:
    webcam.close()
    plt.close(fig)
    print("Camera was closed.")

Of course, it is also possible to apply an image operator before displaying the image:

In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import imageio

try:
    display(HTML("press <kbd>I</kbd>, <kbd>I</kbd> (Kernel Interrupt) to stop the demo!"))
    with imageio.get_reader('<video0>') as webcam:
        fig = plt.figure(figsize=(8,6))
        mpl_image = plt.imshow(webcam.get_next_data())

        while True:
            img = webcam.get_next_data()
            img_processed = 255 - img
            mpl_image.set_data(img_processed)
            fig.canvas.draw()
except KeyboardInterrupt:
    print("Interrupted")
finally:
    webcam.close()
    plt.close(fig)
    print("Camera was closed.")

Matplotlib subplots allow to plot multiple images side by side:
- `subplot(rows, columns, index)`
- creates a grid of plots with shape `rows` x `columns`
- `index` starts with 1, runs horizontally, then vertically
- example: 
  - `subplot(1, 2, 1)` left plot in a grid of 1x2
  - `subplot(1, 2, 2)` right plot in a grid of 1x2

In [ ]:
%matplotlib notebook
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import imageio

try:
    display(HTML("press <kbd>I</kbd>, <kbd>I</kbd> (Kernel Interrupt) to stop the demo!"))
    with imageio.get_reader('<video0>') as webcam:
        fig = plt.figure(figsize=(12,6))
        plt.subplot(1,2,1)
        mpl_image1 = plt.imshow(webcam.get_next_data())
        plt.subplot(1,2,2)
        mpl_image2 = plt.imshow(webcam.get_next_data())

        while True:
            img = webcam.get_next_data()
            img_processed = 255-img
            mpl_image1.set_data(img)
            mpl_image2.set_data(img_processed)
            fig.canvas.draw()
except KeyboardInterrupt:
    print("Interrupted")
finally:
    webcam.close()
    plt.close(fig)
    print("Camera was closed.")